In [1]:
# notebook parameters
id = "factRevenues-Gold"


In [2]:
from common_function import readConfig, absPath, updatePipelineStatus,createHashKey
import pandas as pd
import os


In [3]:
config = readConfig(id)
config

✓ Config loaded successfully: factRevenues-Gold.json


{'pipeline_id': 'factRevenues-Gold',
 'dependency': 'Revenues-Silver',
 'target': {'pk': ['_sk_revenue_id'],
  'format': 'parquet',
  'name': 'factRevenues',
  'path': 'data\\03_gold\\',
  'mode': 'overwrite'}}

In [4]:
# Read data from bronze in delta.
# Meaning - take all data which was loaded to bronze since last successful silver run

path = os.path.join(absPath(), "data\\02_silver\\revenues")

df = pd.read_parquet(path, engine='fastparquet')

In [5]:
# Create hash
df = createHashKey(
    df=df,
    key_columns=['title', 'date'],
    hash_column='_sk_revenue_id'
)

df = createHashKey(
    df=df,
    key_columns=['title'],
    hash_column='_sk_movie'
)

df = createHashKey(
    df=df,
    key_columns=['distributor'],
    hash_column='_sk_distributor'
)



In [6]:
df = df[[
        '_sk_revenue_id',
        '_sk_movie',
        '_sk_distributor',
        'date',
        'revenue', 
        'theaters'
        ]]

In [7]:
target_path = os.path.join(absPath(), config["target"]["path"],config["target"]["name"])

df.to_parquet(target_path, index=False)

In [8]:
updatePipelineStatus(id)

✓ Updated pipeline status: factRevenues-Gold - success
